In [1]:
import polars as pl
import os
import json

In [3]:
df = pl.read_ndjson('../data/mobygames_index.jsonl')
df.head()

id,title,release_date,moby_score,moby_url,description,platforms,genres,developers,publishers,sample_cover_url,sample_screenshot_urls,release_year,error
i64,str,str,f64,str,str,list[str],list[str],list[null],list[null],str,list[str],i64,null
4,"""Sid Meier's Alpha Centauri""","""1999-02-09""",8.0,"""https://www.mobygames.com/game…","""<p>After the 20th century, hum…","[""Windows"", ""Macintosh""]","[""Strategy / tactics"", ""Diagonal-down"", … ""Sci-fi / futuristic""]",[],[],"""https://cdn.mobygames.com/cove…","[""https://cdn.mobygames.com/screenshots/16876257-sid-meiers-alpha-centauri-windows-choosing-a-faction.png"", ""https://cdn.mobygames.com/screenshots/10393989-sid-meiers-alpha-centauri-windows-design-workshop.jpg"", … ""https://cdn.mobygames.com/screenshots/16876134-sid-meiers-alpha-centauri-windows-main-menu.png""]",1999,null
14,"""Chessmaster 5000""","""1996-01-01""",7.6,"""https://www.mobygames.com/game…","""<p><em>Chessmaster 5000</em> i…","[""Windows""]","[""Strategy / tactics"", ""Diagonal-down"", … ""Point and select""]",[],[],"""https://cdn.mobygames.com/cove…","[""https://cdn.mobygames.com/screenshots/7924204-chessmaster-5000-windows-some-game-configuration-options.jpg"", ""https://cdn.mobygames.com/screenshots/7924201-chessmaster-5000-windows-the-opening-book-editor-has-its-own-hel.jpg"", … ""https://cdn.mobygames.com/screenshots/7924343-chessmaster-5000-windows-players-can-create-their-own-board-with.jpg""]",1996,null
13,"""The Chessmaster 4000 Turbo""","""1993-01-01""",8.0,"""https://www.mobygames.com/game…","""<p>The first 32-bit title in t…","[""Windows"", ""Windows 16-bit""]","[""Strategy / tactics"", ""Diagonal-down"", … ""Point and select""]",[],[],"""https://cdn.mobygames.com/cove…","[""https://cdn.mobygames.com/screenshots/1724127-the-chessmaster-4000-turbo-windows-16-bit-a-chinese-board-style.png"", ""https://cdn.mobygames.com/screenshots/1723927-the-chessmaster-4000-turbo-windows-16-bit-a-normal-board-style.png"", … ""https://cdn.mobygames.com/screenshots/1724512-the-chessmaster-4000-turbo-windows-16-bit-a-lot-of-windows.png""]",1993,null
12,"""The Chessmaster 3000""","""1991-01-01""",7.6,"""https://www.mobygames.com/game…","""<p><em>Chessmaster 3000</em> p…","[""DOS"", ""Windows 16-bit"", ""Macintosh""]","[""Strategy / tactics"", ""Diagonal-down"", … ""Point and select""]",[],[],"""https://cdn.mobygames.com/cove…","[""https://cdn.mobygames.com/screenshots/7301801-the-chessmaster-3000-dos-changing-colors.png"", ""https://cdn.mobygames.com/screenshots/11571561-the-chessmaster-3000-macintosh-five-moves-later-its-checkmate.png"", … ""https://cdn.mobygames.com/screenshots/7301774-the-chessmaster-3000-dos-modern-chess-set-2d.png""]",1991,null
1,"""The X-Files Game""","""1998-06-01""",7.1,"""https://www.mobygames.com/game…","""<p>As an extension of one of t…","[""Windows"", ""PlayStation"", ""Macintosh""]","[""Adventure"", ""1st-person"", … ""Licensed""]",[],[],"""https://cdn.mobygames.com/cove…","[""https://cdn.mobygames.com/screenshots/11077669-the-x-files-game-windows-outside-agent-craig-willmores-apartment.jpg"", ""https://cdn.mobygames.com/screenshots/11072410-the-x-files-game-windows-official-fbi-document.jpg"", … ""https://cdn.mobygames.com/screenshots/10471118-the-x-files-game-playstation-exchanging-information-and-acquaint.jpg""]",1998,null


In [5]:
desc_emb = pl.read_ndjson('../embeddings/desc_embeddings.jsonl')
desc_emb.head()

game_id,embedding
i64,list[f64]
1,"[-0.040362, -0.079261, … -0.04446]"
2,"[-0.082043, -0.040053, … -0.015202]"
3,"[-0.094168, -0.073465, … 0.023527]"
4,"[-0.047253, 0.021216, … -0.043057]"
5,"[0.031966, 0.086129, … 0.061764]"


In [8]:
cover_emb = pl.read_ndjson('../embeddings/cover_embeddings.jsonl')
cover_emb.head()

game_id,cover_embedding
str,list[f64]
"""1""","[0.444879, -0.773201, … 0.276423]"
"""2""","[0.723642, 1.28492, … 0.571715]"
"""3""","[-0.032582, 0.503212, … 0.117423]"
"""4""","[-0.132613, -0.102314, … 0.041274]"
"""5""","[2.363587, -0.869756, … -0.236085]"


In [9]:
screenshot_emb = pl.read_ndjson('../embeddings/screenshot_embeddings.jsonl')
screenshot_emb.head()

game_id,screenshot_embedding,num_screenshots
str,list[f64],i64
"""2""","[1.528382, 0.038477, … -0.049169]",5
"""3""","[0.606195, 0.432904, … 0.177745]",5
"""4""","[-0.544538, 0.209987, … 0.829974]",2
"""5""","[0.20944, 0.460073, … 0.050815]",5
"""7""","[-0.945326, 0.329909, … 0.665487]",5


In [7]:
metadata_ids = set(df['id'].to_list())
embedding_ids = set(desc_emb['game_id'].to_list())

print("Metadata only:", len(metadata_ids - embedding_ids))
print("Embeddings only:", len(embedding_ids - metadata_ids))

Metadata only: 90370
Embeddings only: 0


In [10]:
desc_ids = set(desc_emb['game_id'].to_list())
cover_ids = set([int(x) for x in cover_emb['game_id'].to_list()])

print("Covers only:", len(cover_ids - desc_ids))
print("Desc only:", len(desc_ids - cover_ids))
print("Covers & Desc intersection:", len(cover_ids & desc_ids))

Covers only: 78433
Desc only: 41162
Covers & Desc intersection: 56735


In [12]:
screenshot_ids = set([int(x) for x in screenshot_emb['game_id'].to_list()])

print("Screenshots only:", len(screenshot_ids - desc_ids))
print("Desc only:", len(desc_ids - screenshot_ids))
print("Screenshots & Desc intersection:", len(screenshot_ids & desc_ids))

Screenshots only: 1989
Desc only: 52291
Screenshots & Desc intersection: 45606


In [13]:
all_three = desc_ids & cover_ids & screenshot_ids
print("Games with all three embeddings:", len(all_three))

Games with all three embeddings: 21530
